# Modelling coupled degradation mechanisms in PyBaMM

This notebook shows how to set up a PyBaMM model in which many degradation mechanisms run at the same time and interact with one another.

In [1]:
%pip install pybamm -q    # install PyBaMM if it is not installed
import pybamm
import matplotlib.pyplot as plt
import numpy as np


[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


O'Kane et al. modelled four coupled degradation mechanisms: SEI growth, lithium plating, particle cracking and stress-driven loss of active material. The "SEI on cracks" option couples SEI growth and particle cracking by allowing SEI to grow on the cracks. The "partially reversible" option for lithium plating allows the SEI to influence the irreversible component of plating using a function in the OKane2022 parameter file. Particle cracking and stress-driven loss of active material are coupled by default because the stress-strain relations inside the particles are an input for both.

In [2]:
model = pybamm.lithium_ion.DFN(
    {
        "SEI": "solvent-diffusion limited",
        "SEI porosity change": "true",
        "lithium plating": "partially reversible",
        "lithium plating porosity change": "true",  # not required if "SEI porosity change" already set to "true"
        "particle mechanics": ("swelling and cracking", "swelling only"),
        "SEI on cracks": "true",
        "loss of active material": "stress-driven",
    }
)

Depending on the parameter set being used, the particle cracking model can require a large number of mesh points inside the particles to be numerically stable.

In [3]:
param = pybamm.ParameterValues("OKane2022")
var_pts = {
    "x_n": 5,  # negative electrode
    "x_s": 5,  # separator 
    "x_p": 5,  # positive electrode
    "r_n": 30,  # negative particle
    "r_p": 30,  # positive particle
}

Define a cycling protocol and solve. The protocol from O'Kane et al (2022) is used here.

In [4]:
cycle_number = 10
exp = pybamm.Experiment(
    ["Hold at 4.2 V until C/100 (5 minute period)",
    "Rest for 4 hours (5 minute period)",
    "Discharge at 0.1C until 2.5 V (5 minute period)",  # initial capacity check
    "Charge at 0.3C until 4.2 V (5 minute period)",
    "Hold at 4.2 V until C/100 (5 minute period)",]
    + [("Discharge at 1C until 2.5 V",  # ageing cycles
    "Charge at 0.3C until 4.2 V (5 minute period)",
    "Hold at 4.2 V until C/100 (5 minute period)",)] * cycle_number
    + ["Discharge at 0.1C until 2.5 V (5 minute period)"]  # final capacity check
)
sim = pybamm.Simulation(model, parameter_values=param, experiment=exp, var_pts=var_pts)
sol = sim.solve()

At t = 429.694 and h = 8.22097e-12, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 189.694 and h = 9.29241e-12, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 429.055 and h = 1.80513e-11, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 189.056 and h = 1.10586e-11, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 428.452 and h = 1.14514e-11, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 188.452 and h = 7.82525e-12, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 427.875 and h = 1.21404e-11, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 187.875 and h = 1.18816e-11, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 427.318 and h = 1.14594e-11, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 140.057 and h = 8.4999e-10, the corrector convergence failed repeatedly or with |h| = hmin.
A

In [5]:
Qt = sol.summary_variables["Throughput capacity [A.h]"]  # commonly used alternative to cycle number
Q_SEI = sol.summary_variables["Loss of capacity to SEI [A.h]"]
Q_SEI_cr = sol.summary_variables["Loss of capacity to SEi on cracks [A.h]"]
Q_plating = sol.summary_variables["Loss of capacity to lithium plating [A.h]"]
LLI = sol.summary_variables

In [ ]:
plt.rcParams.update({'font.size': 11})
(fig, axs) = plt.subplots(nrows=3,ncols=2,figsize=(13,15))
axs[0,0].plot()